Installing Necessaary libraries

In [11]:
!pip install scikit-learn==1.2.2

  Obtaining dependency information for scikit-learn==1.2.2 from https://files.pythonhosted.org/packages/db/98/169b46a84b48f92df2b5e163fce75d471f4df933f8b3d925a61133210776/scikit_learn-1.2.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/8.3 MB 1.1 MB/s eta 0:00:08
    --------------------------------------- 0.2/8.3 MB 1.7 MB/s eta 0:00:05
   - -------------------------------------- 0.3/8.3 MB 2.0 MB/s eta 0:00:04
   -- ------------------------------------- 0.5/8.3 MB 2.6 MB/s eta 0:00:03
   --- ------------------------------------ 0.7/8.3 MB 2.7 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/8.3 MB 3.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.2/8.3 MB 3.4 MB/s eta 0:00:03
   ------ --------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
farm-haystack 1.25.0 requires scikit-learn>=1.3.0, but you have scikit-learn 1.2.2 which is incompatible.


Importing Necessary libraries

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
import time

In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.nn.functional import cross_entropy

In [3]:
  import torch.nn.functional as F

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

The Dataset contain product_name, Rate , Review, Summary which is descriptive informationn of customer's thought on each product and Sentiment based on Summary. We will use Summary and Sentiment column for this analysis.

In [5]:
df=pd.read_csv("Flipkart_Product_Review.csv")
df.head()

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral


In [6]:
df.shape[0]

205052

In [7]:
df.dtypes

product_name     object
product_price    object
Rate             object
Review           object
Summary          object
Sentiment        object
dtype: object

In [8]:
df.isnull().sum()

product_name         0
product_price        0
Rate                 0
Review           24664
Summary             11
Sentiment            0
dtype: int64

In [9]:
# Convert 'Rate' column to numeric, coercing errors to NaN
df['Rate'] = pd.to_numeric(df['Rate'], errors='coerce')

In [10]:
df.dtypes

product_name      object
product_price     object
Rate             float64
Review            object
Summary           object
Sentiment         object
dtype: object

In [11]:
df.isnull().sum()

product_name         0
product_price        0
Rate                 3
Review           24664
Summary             11
Sentiment            0
dtype: int64

In [12]:
df_rate = df.dropna(subset=['Rate'])

In [13]:
df_rate[(df_rate['Rate']>=3) & (df_rate['Sentiment']=='negative')]

,product_name,product_price,Rate,Review,Summary,Sentiment
53,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,7999,3.0,nice,received dalay 10 days cooler is ok when i was...,negative
120,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,7999,3.0,just okay,not so good,negative
123,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,7999,5.0,perfect product!,not working motors and very bad poor maharaja ...,negative
140,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,7999,3.0,just okay,product quality is very bad and fan blade brok...,negative
179,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,7999,3.0,nice,cooler is good but very dirty smell,negative
...,...,...,...,...,...,...
204998,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3.0,just okay,i expected that this dinner set might be compa...,negative
205012,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5.0,just wow!,plates are small,negative
205014,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3.0,fair,dinner plates is too small,negative
205016,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3.0,fair,small size,negative


In [14]:
df_rate[(df_rate['Rate']<=3) & (df_rate['Sentiment']=='positive')]

,product_name,product_price,Rate,Review,Summary,Sentiment
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3.0,fair,the quality is good but the power of air is de...,positive
7,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3.0,nice,very nice,positive
43,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,7999,3.0,decent product,great deal 65 l quite big tank sufficient cool...,positive
45,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,7999,3.0,very satisfactory,the good things1 very well packed good amount ...,positive
59,MAHARAJA WHITELINE 65 L Desert Air Cooler?????...,7999,1.0,worthless,the product doesnt cool as rated by other cust...,positive
...,...,...,...,...,...,...
204981,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3.0,good,better,positive
204987,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3.0,decent product,it could be better,positive
204989,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3.0,fair,not too much goodthe weight of the set is very...,positive
205011,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,3.0,good,quality is good but size is very small,positive


In [15]:
df_cleaned=df_rate[((df_rate['Rate']>=3) & (df_rate['Sentiment']=='positive')) | ((df_rate['Rate']<=3) & (df_rate['Sentiment']=='negative')) | (df_rate['Sentiment']=='neutral')]

In [16]:
df_cleaned.shape[0]

201333

In [17]:
df_cleaned['Sentiment'].value_counts()

Sentiment
positive    164571
negative     26523
neutral      10239
Name: count, dtype: int64

In [18]:
df_cleaned['product_name'].nunique()

958

In [19]:
df_cleaned.isnull().sum()

product_name         0
product_price        0
Rate                 0
Review           24299
Summary             11
Sentiment            0
dtype: int64

Removing the missing Summary rows from the dataset.

In [20]:
df_cleaned = df_cleaned.dropna(subset=['Summary'])

In [21]:
df_cleaned.shape[0]

201322

In [22]:
df_summary=df_cleaned[['product_name', 'Summary','Sentiment']]
df_summary.head()

,product_name,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,ok ok product,neutral


In [23]:
df_summary.isnull().sum()

product_name    0
Summary         0
Sentiment       0
dtype: int64

In [24]:
df_summary['Sentiment'].value_counts()

Sentiment
positive    164565
negative     26523
neutral      10234
Name: count, dtype: int64

In [25]:

sentiment_counts=df_summary['Sentiment'].value_counts()
total_sentiments = len(df_summary['Sentiment'])

In [26]:
(sentiment_counts / total_sentiments) * 100

Sentiment
positive    81.742184
negative    13.174417
neutral      5.083399
Name: count, dtype: float64

This is a very imbalanced dataset which can cause bias towards one class. So I am going to upsample the Negative and Neutral classes.

Splitting the data in train and test dataset.

In [27]:
X = df_summary['Summary']

y = df_summary['Sentiment']

# Step 3: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Combine the training features and labels into a single DataFrame
train_dataset = pd.DataFrame({'Summary': X_train, 'Sentiment': y_train})

train_dataset.head()

,Summary,Sentiment
10135,quality is nice but go for one larger size,positive
79285,this is the best,positive
86779,very nice product,positive
86846,nice,positive
3819,awesome,positive


In [29]:
# Separate majority and minority classes
df_positive = train_dataset[train_dataset['Sentiment'] == 'positive']
df_negative = train_dataset[train_dataset['Sentiment'] == 'negative']
df_neutral = train_dataset[train_dataset['Sentiment'] == 'neutral']

df_positive=df_positive.sample(n=50000)

# Upsample minority class
df_negative_upsampled = resample(df_negative, 
                                 replace=True,     # sample with replacement
                                 n_samples=50000,    # to match majority class
                                 random_state=42)  # reproducible results

# Upsample minority class
df_neutral_upsampled = resample(df_neutral, 
                                 replace=True,     # sample with replacement
                                 n_samples=50000,    # to match majority class
                                 random_state=42)  # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_positive, df_negative_upsampled, df_neutral_upsampled])

print(df_upsampled['Sentiment'].value_counts())

Sentiment
positive    50000
negative    50000
neutral     50000
Name: count, dtype: int64


From Summary we are making TFIDF word vectors.

In [30]:
# Step 1: Vectorize text data (TF-IDF vectorization)
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(df_upsampled['Summary'])
y_train = df_upsampled['Sentiment']

Used the TFIDF word vectors of Summary and Sentiment to train a random forest model. It took almost 20 minutes to train.

In [31]:
start_time=time.time()

# Step 4: Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

end_time=time.time()

execution_time=end_time-start_time

print("Time taken in minutes:",execution_time/60)

Time taken in minutes: 33.377978920936584


Process the test data and check the predictions.

In [114]:
X_test=vectorizer.transform(X_test)

The evaluation metrics. The f1 score for the positive class is 97%, for the negative class 0.88%. But for the neutral class it's 0.54%.

In [115]:
# Step 5: Predict on test data and evaluate
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.84      0.92      0.88      5265
     neutral       0.49      0.59      0.54      2082
    positive       0.98      0.95      0.97     32918

    accuracy                           0.93     40265
   macro avg       0.77      0.82      0.79     40265
weighted avg       0.94      0.93      0.93     40265



In [116]:
# Step 5: Predict on train data and evaluate
y_pred_train = rf_classifier.predict(X_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

    negative       0.99      0.98      0.99     50000
     neutral       0.97      0.96      0.96     50000
    positive       0.97      0.98      0.98     50000

    accuracy                           0.98    150000
   macro avg       0.98      0.98      0.98    150000
weighted avg       0.98      0.98      0.98    150000



Prediction on given comments.

In [117]:
dict={"comment":["The product was absolutely horrible. I can't believe they can rob me on this with high price and disgusting quality."]}

df_comment = pd.DataFrame(dict)

x = vectorizer.transform(df_comment["comment"])

rf_classifier.predict(x)

array(['positive'], dtype=object)

In [118]:
dict={"comment":["It's Ok product for me."]}

df_comment = pd.DataFrame(dict)

x = vectorizer.transform(df_comment["comment"])

rf_classifier.predict(x)

array(['positive'], dtype=object)

In [119]:
dict={"comment":["I am in love with this dress. Also the brand is known for sustainable clothing so I can spend extra for that cause."]}

df_comment = pd.DataFrame(dict)

x = vectorizer.transform(df_comment["comment"])

rf_classifier.predict(x)

array(['positive'], dtype=object)

In [238]:
# Separate majority and minority classes
df_positive = train_dataset[train_dataset['Sentiment'] == 'positive']
df_negative = train_dataset[train_dataset['Sentiment'] == 'negative']
df_neutral = train_dataset[train_dataset['Sentiment'] == 'neutral']

df_positive=df_positive.sample(n=50)

# Upsample minority class
df_negative_upsampled = resample(df_negative, 
                                 replace=True,     # sample with replacement
                                 n_samples=50,    # to match majority class
                                 random_state=42)  # reproducible results

# # Upsample minority class
# df_neutral_upsampled = resample(df_neutral, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=50,    # to match majority class
#                                  random_state=42)  # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_positive, df_negative_upsampled])

print(df_upsampled['Sentiment'].value_counts())

Sentiment
positive    50
negative    50
Name: count, dtype: int64


In [239]:
sampled_train_df=df_upsampled

In [240]:
# Encode the sentiments
label_encoder = LabelEncoder()
sampled_train_df['label'] = label_encoder.fit_transform(sampled_train_df['Sentiment'])

In [241]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_text(text):
    tokenized_input = tokenizer.encode_plus(text,
        max_length=64,  # Set your desired maximum sequence length
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        return_attention_mask=True)
    return tokenized_input

sampled_train_df["tokenized_input"]=sampled_train_df['Summary'].apply(tokenize_text)

In [242]:
sampled_train_df["features"]=sampled_train_df.apply(lambda row: make_training_data(row['tokenized_input'], row['label']), axis=1)

In [257]:
print(sampled_train_df['Sentiment'].value_counts())

Sentiment
positive    50
negative    50
Name: count, dtype: int64


In [243]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.nn.functional import cross_entropy

start_time=time.time()

# Split data
train_df, val_df = train_test_split(sampled_train_df, test_size=0.2, random_state=42, stratify=sampled_train_df['label'])

train_dataset = train_df['features'].tolist()
val_dataset = val_df['features'].tolist()

# DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
def train(model, train_dataloader, optimizer):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation loop
def evaluate(model, val_dataloader):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            val_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct += (preds == batch['labels']).sum().item()
            total += batch['labels'].size(0)
    accuracy = correct / total
    return val_loss / len(val_dataloader), accuracy

# Train and evaluate the model
num_epochs = 2
for epoch in range(num_epochs):
    train(model, train_dataloader, optimizer)
    val_loss, val_accuracy = evaluate(model, val_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Save the trained model
output_dir = "./SA_model_BERT_Downsample"
model.save_pretrained(output_dir)
    
end_time=time.time()

execution_time=end_time-start_time

print("Execution time in minutes:",execution_time/60)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2, Validation Loss: 0.9291058580080668, Validation Accuracy: 0.5
Epoch 2/2, Validation Loss: 0.8157300353050232, Validation Accuracy: 0.65
Execution time in minutes: 5.719256309668223


In [258]:
print(train_df['Sentiment'].value_counts())

Sentiment
negative    40
positive    40
Name: count, dtype: int64


In [259]:
print(val_df['Sentiment'].value_counts())

Sentiment
positive    10
negative    10
Name: count, dtype: int64


In [283]:
label_names=['negative','positive']

In [286]:
label2id = {k: v for v, k in enumerate(label_names)}
id2label = {v: k for v, k in enumerate(label_names)}
id2label

{0: 'negative', 1: 'positive'}

In [288]:
def get_prediction(Sentence):
    
    inputs = tokenizer(Sentence, return_tensors="pt")
    
    # Step 2: Inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Assuming your TokenClassifierOutput object is named 'output'
    logits = outputs.logits

#     # Apply softmax to get probabilities
#     probs = F.softmax(logits, dim=-1)

    # Get the predicted labels (classes with highest probability)
    predicted_labels = torch.argmax(logits, dim=-1)
    
    predicted_labels = predicted_labels.tolist()

    labels = id2label[predicted_labels[0]]
    return labels

In [289]:
val_df["Predicted Sentiment"]=val_df["Summary"].apply(get_prediction)

In [290]:
print(classification_report(val_df["Sentiment"], val_df["Predicted Sentiment"]))

              precision    recall  f1-score   support

    negative       0.59      1.00      0.74        10
    positive       1.00      0.30      0.46        10

    accuracy                           0.65        20
   macro avg       0.79      0.65      0.60        20
weighted avg       0.79      0.65      0.60        20



In [291]:
train_df["Predicted Sentiment"]=train_df["Summary"].apply(get_prediction)

In [292]:
print(classification_report(train_df["Sentiment"], train_df["Predicted Sentiment"]))

              precision    recall  f1-score   support

    negative       0.75      0.97      0.85        40
    positive       0.96      0.68      0.79        40

    accuracy                           0.82        80
   macro avg       0.86      0.82      0.82        80
weighted avg       0.86      0.82      0.82        80



In [322]:
# Separate majority and minority classes
df_positive = train_dataset[train_dataset['Sentiment'] == 'positive']
df_negative = train_dataset[train_dataset['Sentiment'] == 'negative']
df_neutral = train_dataset[train_dataset['Sentiment'] == 'neutral']

df_positive=df_positive.sample(n=50)

df_negative=df_negative.sample(n=50)

df_neutral=df_neutral.sample(n=50)

# Combine majority class with upsampled minority class
df_downsampled = pd.concat([df_positive, df_negative, df_neutral])

print(df_downsampled['Sentiment'].value_counts())

Sentiment
positive    50
negative    50
neutral     50
Name: count, dtype: int64


In [323]:
sampled_train_df=df_downsampled

In [324]:
# Encode the sentiments
label_encoder = LabelEncoder()
sampled_train_df['label'] = label_encoder.fit_transform(sampled_train_df['Sentiment'])

In [325]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_text(text):
    tokenized_input = tokenizer.encode_plus(text,
        max_length=64,  # Set your desired maximum sequence length
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        return_attention_mask=True)
    return tokenized_input

sampled_train_df["tokenized_input"]=sampled_train_df['Summary'].apply(tokenize_text)

In [326]:
sampled_train_df["features"]=sampled_train_df.apply(lambda row: make_training_data(row['tokenized_input'], row['label']), axis=1)

In [327]:
print(sampled_train_df['Sentiment'].value_counts())

Sentiment
positive    50
negative    50
neutral     50
Name: count, dtype: int64


In [328]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.nn.functional import cross_entropy

start_time=time.time()

# Split data
train_df, val_df = train_test_split(sampled_train_df, test_size=0.2, random_state=42, stratify=sampled_train_df['label'])

train_dataset = train_df['features'].tolist()
val_dataset = val_df['features'].tolist()

# DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
def train(model, train_dataloader, optimizer):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation loop
def evaluate(model, val_dataloader):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            val_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct += (preds == batch['labels']).sum().item()
            total += batch['labels'].size(0)
    accuracy = correct / total
    return val_loss / len(val_dataloader), accuracy

# Train and evaluate the model
num_epochs = 2
for epoch in range(num_epochs):
    train(model, train_dataloader, optimizer)
    val_loss, val_accuracy = evaluate(model, val_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Save the trained model
output_dir = "./SA_model_BERT_Downsample_negposneu"
model.save_pretrained(output_dir)
    
end_time=time.time()

execution_time=end_time-start_time

print("Execution time in minutes:",execution_time/60)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2, Validation Loss: 1.0581064522266388, Validation Accuracy: 0.6
Epoch 2/2, Validation Loss: 1.0078518986701965, Validation Accuracy: 0.5333333333333333
Execution time in minutes: 8.066687500476837


In [331]:
train_df.head()

,Summary,Sentiment,label,tokenized_input,features
34875,very bad,negative,0,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(2200), tens..."
56827,this plazzo is comfortable and nice fabric,positive,2,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(2023), tens..."
128259,average,neutral,1,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(2779), tens..."
102858,pros vibrant easy installation consexpected to...,negative,0,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(4013), tens..."
179864,average,neutral,1,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(2779), tens..."


In [332]:
label_names=['negative','neutral','positive']

In [333]:
label2id = {k: v for v, k in enumerate(label_names)}
id2label = {v: k for v, k in enumerate(label_names)}
id2label

{0: 'negative', 1: 'neutral', 2: 'positive'}

In [334]:
def get_prediction(Sentence):
    
    inputs = tokenizer(Sentence, return_tensors="pt")
    
    # Step 2: Inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Assuming your TokenClassifierOutput object is named 'output'
    logits = outputs.logits

#     # Apply softmax to get probabilities
#     probs = F.softmax(logits, dim=-1)

    # Get the predicted labels (classes with highest probability)
    predicted_labels = torch.argmax(logits, dim=-1)
    
    predicted_labels = predicted_labels.tolist()

    labels = id2label[predicted_labels[0]]
    return labels

In [337]:
val_df["Predicted Sentiment"]=val_df["Summary"].apply(get_prediction)

In [338]:
print(classification_report(val_df["Sentiment"], val_df["Predicted Sentiment"]))

              precision    recall  f1-score   support

    negative       0.88      0.70      0.78        10
     neutral       0.41      0.90      0.56        10
    positive       0.00      0.00      0.00        10

    accuracy                           0.53        30
   macro avg       0.43      0.53      0.45        30
weighted avg       0.43      0.53      0.45        30



C:\Users\Samapti\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Samapti\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Samapti\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
# Separate majority and minority classes
df_positive = train_dataset[train_dataset['Sentiment'] == 'positive']
df_negative = train_dataset[train_dataset['Sentiment'] == 'negative']
df_neutral = train_dataset[train_dataset['Sentiment'] == 'neutral']

df_positive=df_positive.sample(n=3000)

# Upsample minority class
df_negative_upsampled = resample(df_negative, 
                                 replace=True,     # sample with replacement
                                 n_samples=3000,    # to match majority class
                                 random_state=42)  # reproducible results

# # Upsample minority class
# df_neutral_upsampled = resample(df_neutral, 
#                                  replace=True,     # sample with replacement
#                                  n_samples=50,    # to match majority class
#                                  random_state=42)  # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_positive, df_negative_upsampled])

print(df_upsampled['Sentiment'].value_counts())

Sentiment
positive    3000
negative    3000
Name: count, dtype: int64


In [29]:
sampled_train_df=df_upsampled

In [30]:
# Encode the sentiments
label_encoder = LabelEncoder()
sampled_train_df['label'] = label_encoder.fit_transform(sampled_train_df['Sentiment'])

In [33]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_text(text):
    tokenized_input = tokenizer.encode_plus(text,
        max_length=64,  # Set your desired maximum sequence length
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        return_attention_mask=True)
    return tokenized_input

sampled_train_df["tokenized_input"]=sampled_train_df['Summary'].apply(tokenize_text)

In [36]:
def make_training_data(tokenized_input,adjusted_labels):
    input_ids=tokenized_input["input_ids"][0]
    token_type_ids=tokenized_input["token_type_ids"][0]
    attention_mask=tokenized_input["attention_mask"][0]
    
    labels=torch.tensor(adjusted_labels)
    features={
        'input_ids': input_ids,
        'token_type_ids':token_type_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }
    return features

In [37]:
sampled_train_df["features"]=sampled_train_df.apply(lambda row: make_training_data(row['tokenized_input'], row['label']), axis=1)

In [38]:
sampled_train_df.head()

,Summary,Sentiment,label,tokenized_input,features
14375,nice product,positive,1,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(3835), tens..."
185952,good product best quality in affordable price ...,positive,1,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(2204), tens..."
140011,awesome dress my baby so happy thank you flipkart,positive,1,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(12476), ten..."
145401,osm product more soft,positive,1,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(9808), tens..."
22802,i am enjoying using this food processor it rea...,positive,1,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(1045), tens..."


In [39]:
sampled_train_df.shape[0]

6000

In [40]:
print(sampled_train_df['Sentiment'].value_counts())

Sentiment
positive    3000
negative    3000
Name: count, dtype: int64


In [41]:
sampled_train_df.isnull().sum()

Summary            0
Sentiment          0
label              0
tokenized_input    0
features           0
dtype: int64

In [42]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.nn.functional import cross_entropy

start_time=time.time()

# Split data
train_df, val_df = train_test_split(sampled_train_df, test_size=0.2, random_state=42, stratify=sampled_train_df['label'])

train_dataset = train_df['features'].tolist()
val_dataset = val_df['features'].tolist()

# DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
def train(model, train_dataloader, optimizer):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation loop
def evaluate(model, val_dataloader):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            val_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct += (preds == batch['labels']).sum().item()
            total += batch['labels'].size(0)
    accuracy = correct / total
    return val_loss / len(val_dataloader), accuracy

# Train and evaluate the model
num_epochs = 2
for epoch in range(num_epochs):
    train(model, train_dataloader, optimizer)
    val_loss, val_accuracy = evaluate(model, val_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Save the trained model
output_dir = "./SA_model_BERT_Downsample_Big_negpos"
model.save_pretrained(output_dir)
    
end_time=time.time()

execution_time=end_time-start_time

print("Execution time in minutes:",execution_time/60)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2, Validation Loss: 0.09293060516317686, Validation Accuracy: 0.9716666666666667
Epoch 2/2, Validation Loss: 0.08796297380778317, Validation Accuracy: 0.9716666666666667
Execution time in minutes: 304.3290209929148


In [43]:
label_names=['negative','positive']

In [44]:
label2id = {k: v for v, k in enumerate(label_names)}
id2label = {v: k for v, k in enumerate(label_names)}
id2label

{0: 'negative', 1: 'positive'}

In [45]:
def get_prediction(Sentence):
    
    inputs = tokenizer(Sentence, return_tensors="pt")
    
    # Step 2: Inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Assuming your TokenClassifierOutput object is named 'output'
    logits = outputs.logits

#     # Apply softmax to get probabilities
#     probs = F.softmax(logits, dim=-1)

    # Get the predicted labels (classes with highest probability)
    predicted_labels = torch.argmax(logits, dim=-1)
    
    predicted_labels = predicted_labels.tolist()

    labels = id2label[predicted_labels[0]]
    return labels

In [46]:
val_df["Predicted Sentiment"]=val_df["Summary"].apply(get_prediction)

In [47]:
print(classification_report(val_df["Sentiment"], val_df["Predicted Sentiment"]))

              precision    recall  f1-score   support

    negative       0.96      0.99      0.97       600
    positive       0.99      0.95      0.97       600

    accuracy                           0.97      1200
   macro avg       0.97      0.97      0.97      1200
weighted avg       0.97      0.97      0.97      1200



In [48]:
val_df.to_csv("Final_BERT_Predicted.csv")

In [79]:
# Separate majority and minority classes
df_positive = train_dataset[train_dataset['Sentiment'] == 'positive']
df_negative = train_dataset[train_dataset['Sentiment'] == 'negative']
df_neutral = train_dataset[train_dataset['Sentiment'] == 'neutral']

df_positive=df_positive.sample(n=2000)

# Upsample minority class
df_negative_upsampled = resample(df_negative, 
                                 replace=True,     # sample with replacement
                                 n_samples=2000,    # to match majority class
                                 random_state=42)  # reproducible results

# Upsample minority class
df_neutral_upsampled = resample(df_neutral, 
                                 replace=True,     # sample with replacement
                                 n_samples=2000,    # to match majority class
                                 random_state=42)  # reproducible results

# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_positive, df_negative_upsampled,df_neutral_upsampled])

print(df_upsampled['Sentiment'].value_counts())

Sentiment
positive    2000
negative    2000
neutral     2000
Name: count, dtype: int64


In [80]:
sampled_train_df=df_upsampled

In [81]:
# Encode the sentiments
label_encoder = LabelEncoder()
sampled_train_df['label'] = label_encoder.fit_transform(sampled_train_df['Sentiment'])

In [82]:
#Get the values for input_ids, token_type_ids, attention_mask
def tokenize_text(text):
    tokenized_input = tokenizer.encode_plus(text,
        max_length=64,  # Set your desired maximum sequence length
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        return_attention_mask=True)
    return tokenized_input

sampled_train_df["tokenized_input"]=sampled_train_df['Summary'].apply(tokenize_text)

In [83]:
def make_training_data(tokenized_input,adjusted_labels):
    input_ids=tokenized_input["input_ids"][0]
    token_type_ids=tokenized_input["token_type_ids"][0]
    attention_mask=tokenized_input["attention_mask"][0]
    
    labels=torch.tensor(adjusted_labels)
    features={
        'input_ids': input_ids,
        'token_type_ids':token_type_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }
    return features

In [84]:
sampled_train_df["features"]=sampled_train_df.apply(lambda row: make_training_data(row['tokenized_input'], row['label']), axis=1)

In [85]:
sampled_train_df.head()

,Summary,Sentiment,label,tokenized_input,features
145623,perfect,positive,2,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(3819), tens..."
189493,nice,positive,2,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(3835), tens..."
53379,so nice,positive,2,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(2061), tens..."
54998,awesome nice,positive,2,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(12476), ten..."
71118,v good,positive,2,"[input_ids, token_type_ids, attention_mask]","{'input_ids': [tensor(101), tensor(1058), tens..."


In [86]:
sampled_train_df.shape[0]

6000

In [87]:
print(sampled_train_df['Sentiment'].value_counts())

Sentiment
positive    2000
negative    2000
neutral     2000
Name: count, dtype: int64


In [88]:
sampled_train_df.isnull().sum()

Summary            0
Sentiment          0
label              0
tokenized_input    0
features           0
dtype: int64

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.optim import AdamW
from torch.nn.functional import cross_entropy

start_time=time.time()

# Split data
train_df, val_df = train_test_split(sampled_train_df, test_size=0.2, random_state=42, stratify=sampled_train_df['label'])

train_dataset = train_df['features'].tolist()
val_dataset = val_df['features'].tolist()

# DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)

# Load BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
def train(model, train_dataloader, optimizer):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

# Evaluation loop
def evaluate(model, val_dataloader):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_dataloader:
            outputs = model(**batch)
            loss = outputs.loss
            val_loss += loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct += (preds == batch['labels']).sum().item()
            total += batch['labels'].size(0)
    accuracy = correct / total
    return val_loss / len(val_dataloader), accuracy

# Train and evaluate the model
num_epochs = 2
for epoch in range(num_epochs):
    train(model, train_dataloader, optimizer)
    val_loss, val_accuracy = evaluate(model, val_dataloader)
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

# Save the trained model
output_dir = "./SA_model_BERT_Downsample_Big_negposneu"
model.save_pretrained(output_dir)
    
end_time=time.time()

execution_time=end_time-start_time

print("Execution time in minutes:",execution_time/60)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label_names=['negative','neutral','positive']

In [ ]:
label2id = {k: v for v, k in enumerate(label_names)}
id2label = {v: k for v, k in enumerate(label_names)}
id2label

In [ ]:
def get_prediction(Sentence):
    
    inputs = tokenizer(Sentence, return_tensors="pt")
    
    # Step 2: Inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Assuming your TokenClassifierOutput object is named 'output'
    logits = outputs.logits

#     # Apply softmax to get probabilities
#     probs = F.softmax(logits, dim=-1)

    # Get the predicted labels (classes with highest probability)
    predicted_labels = torch.argmax(logits, dim=-1)
    
    predicted_labels = predicted_labels.tolist()

    labels = id2label[predicted_labels[0]]
    return labels

In [ ]:
val_df["Predicted Sentiment"]=val_df["Summary"].apply(get_prediction)

In [ ]:
print(classification_report(val_df["Sentiment"], val_df["Predicted Sentiment"]))

In [341]:
df=pd.read_csv("Predicted_Sentiment.csv")

In [342]:
df["Predicted Sentiment"]=df["Summary"].apply(get_prediction)

In [343]:
print(classification_report(df["Sentiment"], df["Predicted Sentiment"]))

              precision    recall  f1-score   support

    negative       0.27      0.71      0.39       163
     neutral       0.06      0.71      0.10        65
    positive       1.00      0.00      0.00      1022

    accuracy                           0.13      1250
   macro avg       0.44      0.47      0.16      1250
weighted avg       0.86      0.13      0.06      1250

